In [ ]:
import xlwings as xw
import pandas as pd

def load_isin_df():
    """Load ISIN DataFrame from Excel and return it."""
    wb = xw.Book.caller()
    sheet = wb.sheets['Bench']
    df = sheet.range('IsinList').options(pd.DataFrame, header=1, index=False).value
    # get today from "D2" cell as "YYYYMMDD" format
    dt = sheet.range('D2').value
    dt_str = dt.strftime('%Y%m%d')
    df['date'] = int(dt_str)
    # drop if "bench_id" is empty
    df = df[df['bench_id'].notna()]

    # convert issue_date, maturity, list_date, code_date all to "YYYYMMDD" format
    df['issue_date'] = df['issue_date'].apply(lambda x: x if x in (pd.NaT, None) else int(x.strftime('%Y%m%d')))
    df['maturity'] = df['maturity'].apply(lambda x: x if x in (pd.NaT, None) else int(x.strftime('%Y%m%d')))
    df['list_date'] = df['list_date'].apply(lambda x: x if x in (pd.NaT, None) else int(x.strftime('%Y%m%d')))
    df['code_date'] = df['code_date'].apply(lambda x: x if x in (pd.NaT, None) else int(x.strftime('%Y%m%d')))

    # select df that bench_id starts_from KTB_
    ktb_df = df[df['bench_id'].str.startswith('KTB_')].copy()
    # group by bench_id and add column start and end
    # start is issue_date and the end is the None or min of issue dates among bigger than my issue date
    ktb_df['start'] = ktb_df['issue_date']
    # sort by bench_id and issue_date to get the next issue date
    ktb_df.sort_values(['bench_id', 'issue_date'], inplace=True)
    ktb_df['end'] = ktb_df.groupby('bench_id')['issue_date'].shift(-1)

    other_df = df[~df['bench_id'].str.startswith('KTB_')].copy()
    other_df['end'] = other_df['maturity']
    other_df.sort_values(['bench_id', 'maturity'], inplace=True)
    other_df['start'] = other_df.groupby('bench_id')['maturity'].shift(1)
    
    ktb_res = ktb_df[['date', 'isin', 'start', 'end', 'bench_id']].copy()
    other_res = other_df[['date', 'isin', 'start', 'end', 'bench_id']].copy()

    res = pd.concat([ktb_res, other_res], ignore_index=True)

    return res

xw.Book('D:/Projects/marketdata/MarketData.xlsm').set_mock_caller()

df = load_isin_df()
df

,date,isin,start,end,bench_id
0,20250425,KR103502GE63,20240610,20241210,KTB_10Y_BENCH
1,20250425,KR103502GEC4,20241210,NaN,KTB_10Y_BENCH
2,20250425,KR103503GE62,20240610,NaN,KTB_10Y_INFLATION_BENCH
3,20250425,KR103504GE95,20240910,NaN,KTB_20Y_BENCH
4,20250425,KR103503GE39,20240310,20240910,KTB_2Y_BENCH
5,20250425,KR103503GE96,20240910,20250310,KTB_2Y_BENCH
6,20250425,KR103501GF30,20250310,NaN,KTB_2Y_BENCH
7,20250425,KR103502GE97,20240910,20250310,KTB_30Y_BENCH
8,20250425,KR103502GF39,20250310,NaN,KTB_30Y_BENCH
9,20250425,KR103501GE64,20240610,20241210,KTB_3Y_BENCH


In [48]:
df

,prod_type,isin,korean_name,issuer,issue_date,maturity,is_listed,list_date,code_date,bench_id,date
0,채권,KR103503GF38,국고채권 02625-3003(25-3),국고채권,20250310,20300310,상장,20250225,20250219,KTB_5Y_BENCH,20250425
1,채권,KR103502GF39,국고채권 02625-5503(25-2),국고채권,20250310,20550310,상장,20250206,20250124,KTB_30Y_BENCH,20250425
2,채권,KR103501GF30,국고채권 02625-2703(25-1),국고채권,20250310,20270310,상장,20250204,20250123,KTB_2Y_BENCH,20250425
3,채권,KR103502GEC4,국고채권 03000-3412(24-13),국고채권,20241210,20341210,상장,20241022,20241016,KTB_10Y_BENCH,20250425
4,채권,KR103501GEC6,국고채권 02875-2712(24-12),국고채권,20241210,20271210,상장,20241015,20241008,KTB_3Y_BENCH,20250425
5,채권,KR103505GE94,국고채권 02750-7409(24-11),국고채권,20240910,20740910,상장,20240919,20240913,KTB_50Y_BENCH,20250425
6,채권,KR103504GE95,국고채권 02875-4409(24-10),국고채권,20240910,20440910,상장,20240828,20240822,KTB_20Y_BENCH,20250425
7,채권,KR103503GE96,국고채권 02875-2609(24-9),국고채권,20240910,20260910,상장,20240814,20240808,KTB_2Y_BENCH,20250425
8,채권,KR103502GE97,국고채권 02750-5409(24-8),국고채권,20240910,20540910,상장,20240806,20240731,KTB_30Y_BENCH,20250425
9,채권,KR103501GE98,국고채권 03000-2909(24-7),국고채권,20240910,20290910,상장,20240723,20240717,KTB_5Y_BENCH,20250425
